In [23]:
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"
import pandas as pd
import seaborn as sns
sns.set()
import gc
gc.collect()
%matplotlib inline
import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
from IPython.display import clear_output
#set_matplotlib_formats('retina')
#from tqdm import tqdm
#tqdm.pandas()
import numpy as np
from pyhive import presto
from datetime import datetime, timedelta
#from bson import ObjectId
#from functools import reduce
from sklearn.cluster import KMeans
#from pymongo import MongoClient
import glob
import warnings
warnings.filterwarnings('ignore')
from datetime import date
import json
import re
#import dtale
#from h3 import h3
# import pandasql as ps

In [30]:
metabase_connection = presto.connect(
        #host='bi-trino-2.serving.data.production.internal',
        host='bi-trino.serving.data.production.internal',
        #host='bi-presto.serving.data.production.internal',
        #host='prime-trino.serving.data.production.internal',
        #host="presto.processing.yoda.run",
        #host = "processing-2.processing.data.production.internal",
        port=80,
        protocol='http',
        catalog ='hive',
        username='praveen.u@rapido.bike')

In [16]:
# from datetime import datetime
# #yyyymmdd = pd.date_range(start='2023-05-10', end='2023-05-13').strftime('%Y%m%d').to_list()
# yyyymmdd = pd.date_range(start='2023-05-13', end='2023-05-13').strftime('%Y%m%d').to_list()

# df = pd.DataFrame()
# #for city in ["Bangalore","Hyderabad","Delhi","Jaipur","Mumbai","Pune","Chennai"]:
# for city in ["Mumbai"]:
#     for i in range(len(yyyymmdd)):
#         dt = yyyymmdd[i]
#         query1 = f"""
#     select base.city,base.yyyymmdd, substr(hhmm,1,2) as hh,
#     case 
#     when Offline_time <= 0.25 then '1. 0-15 Mins'
#     when Offline_time <= 0.5 then '2. 15-30 Mins'
#     when Offline_time <= 0.75 then '3. 30-45 Mins'
#     when Offline_time <= 1 then '4. 45-60 Mins'
#     when Offline_time <= 1.5 then '5. 1-1_1/2 Hrs'
#     when Offline_time <= 2 then '6. 1_1/2-2 Hrs'
#     when Offline_time <= 3 then '7. 2-3 Hrs'
#     when Offline_time > 3 then '8. >3 Hrs'
#     else '9. NA' end as Offline_time_Flag, 
#     Comp_Day_Affinity,
#     count(distinct base.captain_id) as captain_count

#     -- distinct Next_Status_Flag
#     from(
#         select *,
#         -- ((cast(substr(cast(Next_Epoch as varchar),1,10) as bigint) - cast(substr(cast(epoch as varchar),1,10) as bigint))/(3600.00))
#         case when Next_Epoch is NULL then to_unixtime(cast(REPLACE(cast(date_parse(yyyymmdd, '%Y%m%d') as varchar),'00:00:00.000','21:00:00.000') as timestamp)) else cast(substr(cast(Next_Epoch as varchar),1,10) as bigint) end as Next_Epoch2,
#         case when Next_Status_Flag is NULL then 'NA' else Next_Status_Flag end as Next_Status_Flag2,
#         (((case when Next_Epoch is NULL then to_unixtime(cast(REPLACE(cast(date_parse(yyyymmdd, '%Y%m%d') as varchar),'00:00:00.000','21:00:00.000') as timestamp)) else cast(substr(cast(Next_Epoch as varchar),1,10) as bigint) end) - (cast(substr(cast(epoch as varchar),1,10) as bigint)))/(3600.00)) as Offline_time
#         from(
#             select a.*,
#             LEAD(Status_Flag,1) over (PARTITION BY captain_id ORDER BY epoch) as Next_Status_Flag,
#             LEAD(epoch,1) over (PARTITION BY captain_id,Status_Flag ORDER BY epoch) as Next_Epoch
#             from(
#                     select t1.*, t2.city
#                     from(
#                             select case when status in (2,3,5,6,7,8) then 'Online' else 'Offline' end as Status_Flag,a.*

#                             from  datasets.supplycursory_history a
#                             where yyyymmdd = cast({dt} as varchar) 
#                             -- where yyyymmdd = '20230513' 
#                             and lower(trim(service)) like '%auto%'
#                             -- and captain_id = '637df10983bccc342b905692'
#                             and (cast(substr(hhmm,1,2) as int) >= 8 and cast(substr(hhmm,1,2) as int) <= 20)
#                             and location in 
#                                         (Select distinct hex_id
#                                         from datasets.city_cluster_hex 
#                                         where resolution = 8
#                                         -- and city = 'Chennai'
#                                         and city = '{city}'
#                                         )
#                     ) t1
#                     left join (select distinct hex_id, city from datasets.city_cluster_hex where resolution = 8 and city = '{city}' ) t2
#                     on t1.location = t2.hex_id
#             ) as a

#         ) as b
#         where Status_Flag != case when Next_Status_Flag is NULL then 'NA' else Next_Status_Flag end
#         and Status_Flag = 'Offline'
#     ) as base

#     left join (
#     select captain_id,yyyymmdd, 
#     case 
#     when (Ola_MP_Flag+Ola_AF_Flag+Ola_EP_Flag) >= 2 and (Uber_MP_Flag+Uber_AF_Flag+Uber_EP_Flag) >= 2  then 'Both' 
#     when (Ola_MP_Flag+Ola_AF_Flag+Ola_EP_Flag) >= 2  then 'Ola' 
#     when (Uber_MP_Flag+Uber_AF_Flag+Uber_EP_Flag) >= 2  then 'Uber' else 'NP' end as Comp_Day_Affinity
#     from (
#         select captain_id,yyyymmdd, 
#         sum(case when time_period = 'morning_peak' and Ola_Hour_favorite >=3 then 1 else 0 end) as Ola_MP_Flag,
#         sum(case when time_period = 'afternoon' and Ola_Hour_favorite >=4 then 1 else 0 end) as Ola_AF_Flag,
#         sum(case when time_period = 'evening_peak' and Ola_Hour_favorite >=4 then 1 else 0 end) as Ola_EP_Flag,

#         sum(case when time_period = 'morning_peak' and Uber_Hour_favorite >=3 then 1 else 0 end) as Uber_MP_Flag,
#         sum(case when time_period = 'afternoon' and Uber_Hour_favorite >=4 then 1 else 0 end) as Uber_AF_Flag,
#         sum(case when time_period = 'evening_peak' and Uber_Hour_favorite >=4 then 1 else 0 end) as Uber_EP_Flag

#         from(
#                 select captain_id,yyyymmdd,time_period,sum(Ola_Hour_favorite) as Ola_Hour_favorite,sum(Uber_Hour_favorite) as Uber_Hour_favorite
#                 from(
#                     select captain_id, yyyymmdd, substr(start_hour,1,2) as Hour, time_period,
#                     sum(case when app_name = 'Ola' then Index_Flag else 0 end) as Ola_Hour_favorite,
#                     sum(case when app_name = 'Uber' then Index_Flag else 0 end) as Uber_Hour_favorite
#                     from hive.experiments.Auto_Competitor_App_usage_v2
#                     where yyyymmdd = cast({dt} as varchar) 
#                     --  and yyyymmdd <= '20230501'
#                     and time_period IN ('morning_peak','afternoon','evening_peak') 
#                     and city_name = '{city}'
#                     group by 1,2,3,4
#                 )
#                 group by 1,2,3
#             )
#             group by 1,2
#         ) t1
#     ) day_affinity
#     on base.captain_id = day_affinity.captain_id
#     and base.yyyymmdd = day_affinity.yyyymmdd
#     group by 1,2,3,4,5
#         """
#         print(city,dt)
#         t1 = pd.read_sql_query(query1, metabase_connection)
#         df = pd.concat([t1, df], axis=0).reset_index(drop=True)

Mumbai 20230513


In [14]:
df.shape

(5964, 6)

In [7]:
city = ["Bangalore","Hyderabad","Delhi","Jaipur"]

In [8]:
df2 = df[df['city'].isin(city)]

In [9]:
df2.shape

(7677, 6)

In [17]:
df.to_csv("Bring_Me_Back_On_Duty_Data_test.csv", index=False)

In [9]:
# t1_dt=pd.DataFrame()

# for i in pd.date_range('20230620','20230620'):

#     query="""
#     select
#             *
#     from        
#     (
#         with v0 as 

#             (
#                 select *
#                 from        
#                         (select 
#                                 order_id, yyyymmdd, city_name, order_type, captain_id, pickup_location_hex_8,
#                                 event_type, order_status, spd_fraud_flag, service_obj_service_name as service_name,
#                                 accept_to_pickup_distance, estimated_accept_to_pickup_distance as fm_osrm, eta,
#                                 distance_final_distance, updated_hhmmss, updated_epoch, dispatch_global_priority,
#                                 (case when yyyymmdd <= '20230704' then 'Pre Period' else 'Post Period' end) as period,

#                                 (case when cast(substr(hhmmss,1,2) as real) >= 8 and cast(substr(hhmmss,1,2) as real) <= 11 then 'morning_peak'
#                                 when cast(substr(hhmmss,1,2) as real) > 11 and cast(substr(hhmmss,1,2) as real) < 17 then 'afternoon'
#                                 when cast(substr(hhmmss,1,2) as real) >= 17 and cast(substr(hhmmss,1,2) as real) <= 21 then 'evening_peak'
#                                 when cast(substr(hhmmss,1,2) as real) >= 0 and cast(substr(hhmmss,1,2) as real) < 8 then 'rest_morning'
#                                 else 'Rest_Evening' end) as time_bucket,
#                                 row_number() over (partition by order_id order by updated_epoch desc) as rank_1,

#                                 cardinality(array_remove(array_distinct(array_agg(captain_id) over (partition by order_id)), '')) as captains_pinged,
#                                 array_agg(event_type) over (partition by order_id) as event_type_agg,
#                                 cardinality(array_distinct(array_remove(split(replace(replace(replace(map_riders, '['), ']'), '"'), ','), ''))) as map_riders_count, cancel_reason


#                         from 
#                                 orders.order_logs_immutable
#                         where 
#                                 yyyymmdd = '{dt}'
#                                 and city_name in ('Pune','Hyderabad')
#                                 and service_obj_service_name in ('Auto')
#                         )
#             ),

#         v1 as 
#         (
#             select city_name,test_ctrl, hex_id ,time_bucket
#             from experiments_internal.auto_apr_fm_reduction_exp_230704 
#             where city_name in ('Pune','Hyderabad')

#         ),

#     v2 as 
#         (select 
#             order_id as orderid, 
#             count(distinct rider_id_fc) as captain_count
#         from
#         (
#         select 
#             order_id, rider_id, yyyymmdd, event_type,
#             -- cardinality(array_remove(split(array_join(transform(cast(json_extract(data, '$.filteredCaptains') as array<json>), x->json_extract(x, '$.captain.riderId')), ','), ','), '')) as rider_fc_len,
#             split(array_join(transform(cast(json_extract(data, '$.filteredCaptains') as array<json>), x->json_extract(x, '$.captain.riderId')), ','), ',') as rider_id_fc_list,
#             split(array_join(transform(cast(json_extract(data, '$.filteredCaptains') as array<json>), x->json_extract(x, '$.filterType')), ','), ',') as filterType_list,
#             service_detail_id, propagation_batch_id, updated_epoch,
#             min(updated_epoch) over (partition by order_id, propagation_batch_id) as batch_epoch, updated_hhmmss,
#             other_orders_in_propagation_for_rider, quarter_hour, riders, data
#             -- , city_name, service_name
#         from
#             orders.dispatch_propagation_immutable 
#             -- a
#             -- left join (select city_display_name as city_name, service_level as service_name, service_detail_id as service_detail_id_1
#             -- from datasets.service_mapping) b on a.service_detail_id = b.service_detail_id_1
#         where 
#             yyyymmdd = '{dt}'
#             and service_detail_id in ('61769b50d21e19252c6f60e8','5ef2bc5b85846b775f97d170')
#             and event_type in ('propagation_initiated_for_batch')

#         ) cross join unnest(rider_id_fc_list, filterType_list) as t(rider_id_fc, filterType)
#         where filterType in ('MMO_LIMIT_REACHED')
#         group by 1

#         )

#     select 
#             v0.*, test_ctrl, captain_count
#     from 
#             v0 
#             left join v1 
#             on v0.pickup_location_hex_8=v1.hex_id and v0.time_bucket = v1.time_bucket 
#             left join v2 
#             on v0.order_id = v2.orderid
            
#     )""".format(dt=i.strftime('%Y%m%d'))

#     df_test = pd.read_sql_query(query, metabase_connection)

#     print(i)
#     t1_dt=t1_dt.append(df_test)


In [ ]:
# metabase_connection = presto.connect(
#         #host='bi-trino-2.serving.data.production.internal',
#         #host='bi-trino.serving.data.production.internal',
#         #host='bi-presto.serving.data.production.internal',
#         #host='prime-trino.serving.data.production.internal',
#         host="presto.processing.yoda.run",
#         port=80,
#         protocol='http',
#         catalog ='hive',
#         username='praveen.u@rapido.bike')

In [35]:
t1_dt=pd.DataFrame()

for i in pd.date_range('20230723','20230725'):

    query="""

    with main as (

                    select *,case when rw=1 and offline_to_online_time is not null then 'skip'
                    else 'consider' end as rows_to_consider

                    from
                    (

                    select *, row_number() over(partition by captain_id order by timestamp) rw 
                        from
                        (
                        select city,yyyymmdd,hh as hours,quarter_hour,a.captain_id,timestamp,date_format(from_unixtime(timestamp/1000), '%Y-%m-%d %h:%i:%s') timestamp_readable,
                        case when (stored_status in ('2','3','5','6','7','8') and status = '9') then timestamp end as online_to_offline_time,
                        case when status in ('2','3','5','6','7','8') and stored_status ='9'   then timestamp end as offline_to_online_time
                        from hive.mismatch.rider_detail_changes as a 

                        join (

                                select distinct city,captain_id 
                                from datasets.captain_metrics 
                                where yyyymmdd='{dt}' and city in ('Delhi') and service_mode = 'auto'

                            ) b 
                                  on a.captain_id = b.captain_id

                        where yyyymmdd='{dt}' and type='status'  and hh in ('08','09','10','11','12','13','17','18','19','20','21','22') 

                        ) 
                        where online_to_offline_time is not null or  offline_to_online_time is not null 

                        order by timestamp
                    )
                ),



    table1 as 

            (
            select city,yyyymmdd,hours,quarter_hour,captain_id,online_to_offline_time,
            concat(captain_id,'_',cast(row_number() over(partition by captain_id order by online_to_offline_time) as varchar )) as unique_key

            from main where rows_to_consider='consider' and online_to_offline_time is not null 
          ),

     table2 as 
            (
            select city,yyyymmdd,quarter_hour,captain_id,offline_to_online_time,
            concat(captain_id,'_',cast(row_number() over(partition by captain_id order by offline_to_online_time) as varchar )) as unique_key

            from main where rows_to_consider='consider' and offline_to_online_time is not null
            ),


    offline_caps as 
            (

            select city,yyyymmdd,hours,count(distinct captain_id) as offline_captains
            from main where online_to_offline_time is not null 
            group by 1,2,3 
            ),
    online_caps as 

            (
            select city,yyyymmdd,hours,count(distinct captain_id) as online_captains
            from main where offline_to_online_time is not null 
            group by 1,2,3 
            ),

    new_one as
            ( 
                select * 
                from
                (
                    select a.city,a.yyyymmdd,a.hours,a.captain_id,online_to_offline_time,date_format(from_unixtime(online_to_offline_time/1000), '%Y-%m-%d %h:%i:%s') readable_online_to_offline_time,
                        offline_to_online_time,date_format(from_unixtime(offline_to_online_time/1000), '%Y-%m-%d %h:%i:%s') readable_offline_to_online_time,
                        a.unique_key,
                        (offline_to_online_time-online_to_offline_time)/60000 as mins

                        from table1 as a 

                        left join table2 as b 

                        on a.yyyymmdd=b.yyyymmdd and a.captain_id=b.captain_id and a.unique_key = b.unique_key
                        order by unique_key  
                )    
                 where mins is not null and mins>=1 

            ),

    bring_back  as 
                (
                    select distinct yyyymmdd,city,hh,epoch,
                        date_format(from_unixtime(epoch/1000), '%Y-%m-%d %h:%i:%s') readable_epoch,
                        event_props_user_id as caps_clicked_bring_back
                        FROM
                        (
                        SELECT 
                          yyyymmdd, epoch,
                          hh,
                          event_props_city as city,
                          event_props_user_id

                        FROM 
                        clevertap.captain_bring_back_immutable
                        WHERE yyyymmdd='{dt}' and event_props_city = 'Delhi'

                )
                where hh in ('08','09','10','11','12','13','17','18','19','20','21','22')


            ),

      system_trigger as (

        select yyyymmdd,city,hh,count(distinct caps_clicked_bring_back) as brought_back_by_system_in_20mins
        from
        (
            select distinct k1.yyyymmdd,k1.city,k1.hh,epoch,
            date_format(from_unixtime(epoch/1000), '%Y-%m-%d %h:%i:%s') readable_epoch,
            k1.event_props_user_id as caps_clicked_bring_back,trigger_epoch,trigger_readable_epoch,

            row_number() over(partition by k1.event_props_user_id,trigger_readable_epoch order by epoch desc) as rw_num

                FROM
                (
                SELECT 
                
                yyyymmdd, epoch,hh,event_props_city as city,event_props_user_id

                FROM 
                clevertap.captain_bring_back_immutable 
                WHERE 
                 yyyymmdd='{dt}' and event_props_city = 'Delhi'

                ) as k1 

            left join 
            (

            select  yyyymmdd,hh,event_props_city,epoch as trigger_epoch,
                    date_format(from_unixtime(epoch/1000), '%Y-%m-%d %h:%i:%s') trigger_readable_epoch,
                    JSON_EXTRACT_SCALAR(event_props, '$.trigger') eventProps_trigger,event_props_user_id

                    from  clevertap.captain_online_immutable 
                    where yyyymmdd='{dt}' and lower(event_props_city) in ('delhi') and hh in ('08','09','10','11','12','13','17','18','19','20','21','22')
                    and JSON_EXTRACT_SCALAR(event_props, '$.trigger')='system'
                    order by epoch 
            ) as k2 

            on k1.yyyymmdd =k2.yyyymmdd and k1.hh =k2.hh and k1.event_props_user_id = k2.event_props_user_id

            where k1.hh in ('08','09','10','11','12','13','17','18','19','20','21','22') and trigger_epoch>epoch

            order by k1.hh,k1.event_props_user_id,epoch

            ) where rw_num=1 

            group by 1,2,3 
            ),


    final_one as
    (            select city,yyyymmdd,hh,
                count(distinct caps_clicked_bring_back) as caps_clicked,
                count(caps_clicked_bring_back) as no_of_times_clicked,
                count( case when mins<5 then captain_id end) as online_before_5mins,
                count( case when mins>=5 and mins<10 then  captain_id end) as online_before_10mins,
                count( case when mins>=10 and mins<15 then  captain_id end) as online_before_15mins,   
                count( case when mins>=15 and mins<19 then  captain_id end) as online_before_19mins,
                count( case when mins>=19 and mins<21 then  captain_id end) as online_around_20mins,
                count( case when mins>=21 then  captain_id end) as online_after_20mins

            FROM
            (

            select v1.yyyymmdd,v1.city,v1.hh,captain_id,caps_clicked_bring_back,online_to_offline_time,readable_online_to_offline_time,offline_to_online_time,readable_offline_to_online_time,mins,epoch,readable_epoch,
            row_number() over(partition by captain_id,epoch order by epoch,offline_to_online_time) as rw_num

            FROM bring_back v1 

            left join new_one v2 

            on v1.city = v2.city and v1.yyyymmdd =v2.yyyymmdd and v1.caps_clicked_bring_back = v2.captain_id and v1.hh =v2.hours

            where offline_to_online_time>epoch -- to make sure we are checking post clicking bring back 

            order by hh,captain_id,epoch,offline_to_online_time

            )

            where rw_num =1 

            group by 1,2,3 

      )

      select p1.city,p1.yyyymmdd,p1.hh,offline_captains,online_captains,caps_clicked,no_of_times_clicked,online_before_5mins,online_before_10mins,online_before_15mins,online_before_19mins,brought_back_by_system_in_20mins,
      no_of_times_clicked - (online_before_5mins+online_before_10mins+online_before_15mins+online_before_19mins+brought_back_by_system_in_20mins) as not_brought_back_by_system

      from final_one p1 
      left join  system_trigger as p2 

        on p1.city = p2.city and p1.yyyymmdd = p2.yyyymmdd and p1.hh = p2.hh 

      left join offline_caps as t2 
        on p1.yyyymmdd =t2.yyyymmdd and p1.city = t2.city and p1.hh = t2.hours
     left join online_caps as t3
        on p1.yyyymmdd =t3.yyyymmdd and p1.city = t3.city and p1.hh = t3.hours


    """.format(dt=i.strftime('%Y%m%d'))

    df_test = pd.read_sql_query(query, metabase_connection)

    print(i)
    t1_dt=t1_dt.append(df_test)


2023-07-23 00:00:00
2023-07-24 00:00:00
2023-07-25 00:00:00


In [36]:
t1_dt.to_clipboard()

In [ ]:
### for getting any addition of rides or 

In [ ]:
t1_dt_one=pd.DataFrame()

for i in pd.date_range('20230723','20230723'):

    query="""with data1 as
        ( 
        select city,captain_id,avg(total_login_hours) as pre_avg_total_login_hours,
        avg(morning_peak_lhrs) as pre_morning_peak_lhrs,
        avg(Evening_Peak_lhrs) as pre_Evening_Peak_lhrs,
        avg(net_rides_taxi) as pre_RPR,
        avg(morning_peak_net_rides) as pre_morning_peak_rpr,
        avg(evening_peak_net_rides) as pre_evening_peak_rpr

        from
        (

         select yyyymmdd,city,captain_id,total_login_hours,
            (total_login_hour_08+ total_login_hour_09+ total_login_hour_10+total_login_hour_11+total_login_hour_12) as morning_peak_lhrs,
            (total_login_hour_17+total_login_hour_18+total_login_hour_19+total_login_hour_20+total_login_hour_21) as  Evening_Peak_lhrs,
            net_rides_taxi,
            (net_rides_taxi_hour_08+net_rides_taxi_hour_09+net_rides_taxi_hour_10+net_rides_taxi_hour_11+net_rides_taxi_hour_12) as morning_peak_net_rides,
            (net_rides_taxi_hour_17+net_rides_taxi_hour_18+net_rides_taxi_hour_19+net_rides_taxi_hour_20+net_rides_taxi_hour_21) as evening_peak_net_rides



                from datasets.captain_metrics 
                where yyyymmdd>='20230703' and yyyymmdd<='20230717'  and service_mode='auto'
                and lower(trim(city)) in ('bangalore','delhi')
        )
        group by 1,2 
        ),

    tab2 as 
        (

        select yyyymmdd,city,caps_clicked_bring_back,post_avg_total_login_hours,post_morning_peak_lhrs,
        post_Evening_Peak_lhrs,post_RPR,post_morning_peak_rpr,post_evening_peak_rpr

        from
        (
         select distinct yyyymmdd,city,caps_clicked_bring_back,
            from
            (
                select distinct k1.yyyymmdd,k1.city,k1.hh,epoch,
                date_format(from_unixtime(epoch/1000), '%Y-%m-%d %h:%i:%s') readable_epoch,
                k1.event_props_user_id as caps_clicked_bring_back,trigger_epoch,trigger_readable_epoch,

                row_number() over(partition by k1.event_props_user_id,trigger_readable_epoch order by epoch desc) as rw_num

                    FROM
                    (
                    SELECT 

                    yyyymmdd, epoch,hh,event_props_city as city,event_props_user_id

                    FROM 
                    clevertap.captain_bring_back_immutable 
                    WHERE 
                     yyyymmdd='{dt}' and event_props_city = 'Delhi'

                    ) as k1 

                left join 
                (

                select  yyyymmdd,hh,event_props_city,epoch as trigger_epoch,
                        date_format(from_unixtime(epoch/1000), '%Y-%m-%d %h:%i:%s') trigger_readable_epoch,
                        JSON_EXTRACT_SCALAR(event_props, '$.trigger') eventProps_trigger,event_props_user_id

                        from  clevertap.captain_online_immutable 
                        where yyyymmdd='{dt}' and lower(event_props_city) in ('delhi') and hh in ('08','09','10','11','12','13','17','18','19','20','21','22')
                        and JSON_EXTRACT_SCALAR(event_props, '$.trigger')='system'
                        order by epoch 
                ) as k2 

                on k1.yyyymmdd =k2.yyyymmdd and k1.hh =k2.hh and k1.event_props_user_id = k2.event_props_user_id

                where k1.hh in ('08','09','10','11','12','13','17','18','19','20','21','22') and trigger_epoch>epoch

                order by k1.hh,k1.event_props_user_id,epoch
                )
                where rw_num=1 

            ) v1

                left join
                (
                select city,captain_id,avg(total_login_hours) as post_avg_total_login_hours,
                avg(morning_peak_lhrs) as post_morning_peak_lhrs,
                avg(Evening_Peak_lhrs) as post_Evening_Peak_lhrs,
                avg(net_rides_taxi) as post_RPR,
                avg(morning_peak_net_rides) as post_morning_peak_rpr,
                avg(evening_peak_net_rides) as post_evening_peak_rpr

                from
                (

                 select yyyymmdd,city,captain_id,total_login_hours,
                    (total_login_hour_08+ total_login_hour_09+ total_login_hour_10+total_login_hour_11+total_login_hour_12) as morning_peak_lhrs,
                    (total_login_hour_17+total_login_hour_18+total_login_hour_19+total_login_hour_20+total_login_hour_21) as  Evening_Peak_lhrs,
                    net_rides_taxi,
                    (net_rides_taxi_hour_08+net_rides_taxi_hour_09+net_rides_taxi_hour_10+net_rides_taxi_hour_11+net_rides_taxi_hour_12) as morning_peak_net_rides,
                    (net_rides_taxi_hour_17+net_rides_taxi_hour_18+net_rides_taxi_hour_19+net_rides_taxi_hour_20+net_rides_taxi_hour_21) as evening_peak_net_rides



                        from datasets.captain_metrics 
                        where yyyymmdd='{dt}' and service_mode='auto'
                        and lower(trim(city)) in ('delhi')
                )
                group by 1,2 


                ) as v2

            on v1.yyyymmdd = v2.yyyymmdd and v1.caps_clicked_bring_back = v2.captain_id

    )

    select
    tab2.yyyymmdd,tab2.city,caps_clicked_bring_back,
    pre_avg_total_login_hours,pre_morning_peak_lhrs,pre_Evening_Peak_lhrs,pre_RPR,pre_morning_peak_rpr,pre_evening_peak_rpr
    post_avg_total_login_hours,post_morning_peak_lhrs,post_Evening_Peak_lhrs,post_RPR,post_morning_peak_rpr,post_evening_peak_rpr
    from 
    tab2
    join data1

    on tab2.city = data1.city and tab2.captain_id = data1.caps_clicked_bring_back


    """.format(dt=i.strftime('%Y%m%d'))

    df_test = pd.read_sql_query(query, metabase_connection)

    print(i)
    t1_dt_one=t1_dt_one.append(df_test)
